In [1]:
import os
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.losses import BinaryCrossentropy

import seaborn as sns
import helpers
import dataset
import test_lstm
import tensorflow as tf
import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

2022-12-02 02:38:12.631113: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
lie_trial_path = './processed_lie/' #60 entries
truth_trial_path = './processed_truth/' #61 entries

In [3]:
X, Y = dataset.LSTM_preprocessing(truth_trial_path, lie_trial_path)

In [5]:
TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

In [6]:
LSTM_NEURONS = 32
DROPOUT = 0
EPOCHS = 10
OPTIM = 'adam'
# LOSS = 'binary_crossentropy'
LOSS = BinaryCrossentropy(from_logits=False)

n_inputRows = xTrain.shape[0] 
n_timesteps = xTrain.shape[1]
n_features = xTrain.shape[2]

def LSTM_Model(neuro, drop, loss, optim, step, feat):
  model = Sequential()
  model.add(LSTM(neuro, return_sequences=False, batch_input_shape=(None, step, feat)))

  model.add(Dropout(drop))
  
  model.add(Dense(1, activation='sigmoid')) #softmax

  model.compile(loss=loss, optimizer=optim, metrics=['accuracy'])
  return model

MODEL = LSTM_Model(LSTM_NEURONS, DROPOUT, LOSS, OPTIM, n_timesteps, n_features)
MODEL.summary()
MODEL.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=EPOCHS, verbose=1)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                6016      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 6,049
Trainable params: 6,049
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1536/1536 [==============================] - 9s 4ms/step - loss: 0.5743 - accuracy: 0.7287 - val_loss: 0.5436 - val_accuracy: 0.7559
Epoch 2/10
1536/1536 [==============================] - 7s 4ms/step - loss: 0.5130 - accuracy: 0.7703 - val_loss: 0.4877 - val_accuracy: 0.7791
Epoch 3/10
1536/1536 [========

In [7]:
print(MODEL.predict(xTrain[50:70]))
print(yTrain[50:70])

1/1 [==============================] - 0s 338ms/step
[[0.962403  ]
 [0.7355383 ]
 [0.618671  ]
 [0.9930466 ]
 [0.00355215]
 [0.98420596]
 [0.6331048 ]
 [0.24471064]
 [0.9520015 ]
 [0.73556817]
 [0.73559356]
 [0.24733123]
 [0.73556656]
 [0.978551  ]
 [0.7337395 ]
 [0.5290912 ]
 [0.45830733]
 [0.99705833]
 [0.99303675]
 [0.7355606 ]]
[1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1]


In [8]:
import importlib
importlib.reload(test_lstm)

lie_test_path = './test/Deceptive/processed/'

test_lstm.perdictSingleVideo(lie_test_path, MODEL)

     Unnamed: 0  frame  face_id  timestamp  confidence  success  gaze_0_x  \
0             0      1        0      0.000        0.00        0  0.000000   
1             1      2        0      0.035        0.00        0  0.000000   
2             2      3        0      0.069        0.00        0  0.000000   
3             3      4        0      0.104        0.00        0  0.000000   
4             4      5        0      0.139        0.00        0  0.000000   
..          ...    ...      ...        ...         ...      ...       ...   
426         426    427        0     14.783        0.98        1  0.158218   
427         427    428        0     14.818        0.98        1  0.156531   
428         428    429        0     14.853        0.98        1  0.151447   
429         429    430        0     14.887        0.98        1  0.160341   
430         430    431        0     14.922        0.98        1  0.167285   

     gaze_0_y  gaze_0_z  gaze_1_x  ...  AU25_c  AU26_c  AU28_c  AU45_c  \
0